In [1]:
#importando pacotes úteis
import pandas as pd
import numpy as np
from sklearn.model_selection import LeaveOneOut
from sklearn.svm import SVC, LinearSVC

In [2]:
dt_2apli_ruido_nominal = pd.read_csv('2apli_50ruido_50incerteza.csv');
dt_2apli_ruido_nominal

,App1 - SLLC,App1 - DRAM,App1 - NET,App2 - SLLC,App2 - DRAM,App2 - NET,Nivel de interferencia
0,Muito alto,Muito baixo,Muito baixo,Muito alto,Muito baixo,Muito baixo,Alto
1,Muito alto,Muito baixo,Muito baixo,Medio,Muito baixo,Muito baixo,Baixo
2,Muito alto,Baixo,Muito baixo,Baixo,Muito alto,Muito baixo,Muito baixo
3,Muito alto,Muito baixo,Muito baixo,Baixo,Muito alto,Muito baixo,Baixo
4,Muito alto,Muito baixo,Muito baixo,Baixo,Baixo,Muito alto,Muito baixo
...,...,...,...,...,...,...,...
221,Baixo,Baixo,Muito baixo,Muito alto,Muito baixo,Muito baixo,Muito baixo
222,Muito baixo,Muito baixo,Muito baixo,Muito baixo,Muito baixo,Muito baixo,Muito baixo
223,Muito baixo,Muito baixo,Muito baixo,Muito baixo,Muito baixo,Medio,Muito baixo
224,Muito baixo,Muito baixo,Muito baixo,Muito baixo,Muito baixo,Medio,Muito baixo


In [3]:
dt_2apli_ruido_nominal['Nivel de interferencia'].value_counts()

Muito baixo    120
Baixo           68
Medio           28
Alto             6
Muito alto       4
Name: Nivel de interferencia, dtype: int64

In [4]:
erro_majoritario = dt_2apli_ruido_nominal['Nivel de interferencia'].value_counts().max()/dt_2apli_ruido_nominal['Nivel de interferencia'].count()
print("erro_majoritario = ", erro_majoritario) 

erro_majoritario =  0.5309734513274337


In [5]:
dict_cat = {
    'Muito baixo':0,
    'Baixo':1,
    'Medio':2,
    'Alto':3,
    'Muito alto': 4
}

In [6]:
#O Random Forest só funciona com valores numéricos
dt_2apli_ruido = pd.DataFrame(columns=['App1 - SLLC', 'App1 - DRAM', 'App1 - NET', 'App2 - SLLC', 'App2 - DRAM', 'App2 - NET'])
dt_2apli_ruido['App1 - SLLC'] = dt_2apli_ruido_nominal['App1 - SLLC'].map(dict_cat)
dt_2apli_ruido['App1 - DRAM'] = dt_2apli_ruido_nominal['App1 - DRAM'].map(dict_cat)
dt_2apli_ruido['App1 - NET'] = dt_2apli_ruido_nominal['App1 - NET'].map(dict_cat)
dt_2apli_ruido['App2 - SLLC'] = dt_2apli_ruido_nominal['App2 - SLLC'].map(dict_cat)
dt_2apli_ruido['App2 - DRAM'] = dt_2apli_ruido_nominal['App2 - DRAM'].map(dict_cat)
dt_2apli_ruido['App2 - NET'] = dt_2apli_ruido_nominal['App2 - NET'].map(dict_cat)
dt_category = dt_2apli_ruido_nominal['Nivel de interferencia'].map(dict_cat)
dt_target = np.array(dt_category)
dt_data = np.array(dt_2apli_ruido)
print('dt_target = ', dt_target)
print('dt_data = ', dt_data)

dt_target =  [3 1 0 1 0 2 4 2 1 1 0 0 0 1 2 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 0 0 0 0 0 3 2
 1 1 4 2 1 2 1 1 2 1 0 1 0 2 2 1 1 2 1 2 1 0 0 1 0 1 1 0 0 0 1 2 1 0 1 1 3
 2 1 1 3 4 1 0 1 0 2 4 2 1 2 1 2 3 2 0 2 0 2 1 1 0 0 0 1 2 2 0 1 0 1 1 0 0
 0 1 0 0 0 0 0 0 1 1 1 1 0 1 0 0 0 0 1 0 0 1 0 1 2 1 1 1 1 1 1 0 1 0 0 0 0
 0 1 1 0 0 1 0 1 0 1 2 1 0 1 0 2 2 1 1 2 1 3 2 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0]
dt_data =  [[4 0 0 4 0 0]
 [4 0 0 2 0 0]
 [4 1 0 1 4 0]
 ...
 [0 0 0 0 0 2]
 [0 0 0 0 0 2]
 [0 0 0 0 0 0]]


In [7]:
loo = LeaveOneOut()
loo.get_n_splits(dt_data)
count_rights = 0
count_wrongs = 0
for train_index, test_index in loo.split(dt_data):
    print("Test_index = ", test_index)
    dt_data_train = dt_data[train_index]
    dt_target_train = dt_target[train_index]
    dt_data_test = dt_data[test_index]
    dt_target_test = dt_target[test_index]
    svc_linear = SVC(kernel='linear')
    
    svc_linear.fit(dt_data_train, dt_target_train)
        
    dt_z_test = svc_linear.predict(dt_data_test)
    
    if(dt_z_test == dt_target_test):
        #print("acertou")
        count_rights = count_rights + 1
    else:
        #print("errou")
        count_wrongs = count_wrongs + 1

Test_index =  [0]
Test_index =  [1]
Test_index =  [2]
Test_index =  [3]
Test_index =  [4]
Test_index =  [5]
Test_index =  [6]
Test_index =  [7]
Test_index =  [8]
Test_index =  [9]
Test_index =  [10]
Test_index =  [11]
Test_index =  [12]
Test_index =  [13]
Test_index =  [14]
Test_index =  [15]
Test_index =  [16]
Test_index =  [17]
Test_index =  [18]
Test_index =  [19]
Test_index =  [20]
Test_index =  [21]
Test_index =  [22]
Test_index =  [23]
Test_index =  [24]
Test_index =  [25]
Test_index =  [26]
Test_index =  [27]
Test_index =  [28]
Test_index =  [29]
Test_index =  [30]
Test_index =  [31]
Test_index =  [32]
Test_index =  [33]
Test_index =  [34]
Test_index =  [35]
Test_index =  [36]
Test_index =  [37]
Test_index =  [38]
Test_index =  [39]
Test_index =  [40]
Test_index =  [41]
Test_index =  [42]
Test_index =  [43]
Test_index =  [44]
Test_index =  [45]
Test_index =  [46]
Test_index =  [47]
Test_index =  [48]
Test_index =  [49]
Test_index =  [50]
Test_index =  [51]
Test_index =  [52]
Tes

In [8]:
print("Acertos = ", count_rights)
print("Erros = ", count_wrongs)

Acertos =  139
Erros =  87


In [9]:
n_elements = count_rights + count_wrongs
precision = count_rights/n_elements
print("n_elements = ", n_elements)
print("precision = ", precision)
#precision = np.true_divide(count_rights,215)
#precision 

n_elements =  226
precision =  0.6150442477876106


In [10]:
loo = LeaveOneOut()
loo.get_n_splits(dt_data)
count_rights = 0
count_wrongs = 0
for train_index, test_index in loo.split(dt_data):
    print("Test_index = ", test_index)
    dt_data_train = dt_data[train_index]
    dt_target_train = dt_target[train_index]
    dt_data_test = dt_data[test_index]
    dt_target_test = dt_target[test_index]
    linear_svc = LinearSVC(dual=False)
    
    linear_svc.fit(dt_data_train, dt_target_train)
        
    dt_z_test = linear_svc.predict(dt_data_test)
    
    if(dt_z_test == dt_target_test):
        #print("acertou")
        count_rights = count_rights + 1
    else:
        #print("errou")
        count_wrongs = count_wrongs + 1

Test_index =  [0]
Test_index =  [1]
Test_index =  [2]
Test_index =  [3]
Test_index =  [4]
Test_index =  [5]
Test_index =  [6]
Test_index =  [7]
Test_index =  [8]
Test_index =  [9]
Test_index =  [10]
Test_index =  [11]
Test_index =  [12]
Test_index =  [13]
Test_index =  [14]
Test_index =  [15]
Test_index =  [16]
Test_index =  [17]
Test_index =  [18]
Test_index =  [19]
Test_index =  [20]
Test_index =  [21]
Test_index =  [22]
Test_index =  [23]
Test_index =  [24]
Test_index =  [25]
Test_index =  [26]
Test_index =  [27]
Test_index =  [28]
Test_index =  [29]
Test_index =  [30]
Test_index =  [31]
Test_index =  [32]
Test_index =  [33]
Test_index =  [34]
Test_index =  [35]
Test_index =  [36]
Test_index =  [37]
Test_index =  [38]
Test_index =  [39]
Test_index =  [40]
Test_index =  [41]
Test_index =  [42]
Test_index =  [43]
Test_index =  [44]
Test_index =  [45]
Test_index =  [46]
Test_index =  [47]
Test_index =  [48]
Test_index =  [49]
Test_index =  [50]
Test_index =  [51]
Test_index =  [52]
Tes

In [11]:
print("Acertos = ", count_rights)
print("Erros = ", count_wrongs)

Acertos =  128
Erros =  98


In [12]:
n_elements = count_rights + count_wrongs
precision = count_rights/n_elements
print("n_elements = ", n_elements)
print("precision = ", precision)
#precision = np.true_divide(count_rights,215)
#precision 

n_elements =  226
precision =  0.5663716814159292
